In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv', index_col=0)
df.head()

In [ ]:
df[['cat'+str(i) for i in range(19)]] = df[['cat'+str(i) for i in range(19)]].astype('category')
df.info()

In [ ]:
df_1 = df.drop('cat10', axis=1)

In [ ]:
df_2 = pd.get_dummies(df_1, columns=['cat'+str(i) for i in range(19) if i != 10], drop_first=True)
df_2.head()

In [ ]:
X = df_2.drop('target', axis=1)
y = df_2['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train = np.array(X_train.T)
X_test = np.array(X_test.T)
y_train = np.array(y_train.T).reshape((1, y_train.shape[0]))
y_test = np.array(y_test.T).reshape((1, y_test.shape[0]))

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
y_train.shape[0] 

In [ ]:
def random_mini_batches(X, Y, minibatch_size, seed):
    np.random.seed(seed)
    indices = np.arange(0, X.shape[1])
    np.random.shuffle(indices)
    minibatches = []
    num_of_minibatches = int(X.shape[1] / minibatch_size)
    for i in range(num_of_minibatches):
        X_mini = X[:, indices[i*minibatch_size : minibatch_size*(i+1)]]
        Y_mini = Y[:, indices[i*minibatch_size : minibatch_size*(i+1)]]
        minibatches.append((X_mini, Y_mini))
    return minibatches

In [ ]:
minibatches = random_mini_batches(X_train, y_train, 1000, 1)
print(len(minibatches))
print(minibatches[0][0].shape, minibatches[0][1].shape)

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 15, minibatch_size = 1000, print_cost = True):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X_train -- training set, of shape (input size = 317, number of training examples = 210000)
    Y_train -- test set, of shape (output size = 1, number of training examples = 210000)
    X_test -- training set, of shape (input size = 317, number of training examples = 90000)
    Y_test -- test set, of shape (output size = 1, number of test examples = 90000)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    cost_per_iteration = []                           # Ф-ция потерь на каждой итерации
    
    # Create Placeholders of shape (n_x, n_y)
    ### START CODE HERE ### (1 line)
    X = tf.placeholder(tf.float32, [n_x, None])
    Y = tf.placeholder(tf.float32, [n_y, None])
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    W1 = tf.get_variable("W1", [25,317], initializer = tf.glorot_uniform_initializer())
    b1 = tf.get_variable("b1", [25,1], initializer = tf.zeros_initializer())
    W2 = tf.get_variable("W2", [12,25], initializer = tf.glorot_uniform_initializer())
    b2 = tf.get_variable("b2", [12,1], initializer = tf.zeros_initializer())
    W3 = tf.get_variable("W3", [1,12], initializer = tf.glorot_uniform_initializer())
    b3 = tf.get_variable("b3", [1,1], initializer = tf.zeros_initializer())
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    Z1 = tf.add(tf.matmul(W1, X), b1)                                # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                              # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)                               # Z2 = np.dot(W2, A1) + b2
    A2 = tf.nn.relu(Z2)                                              # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)   
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    # print('logits:', logits, 'labels:', labels)
    
    #!!!!!!!!!!!!!!!!!!Переделать на one_hot
    cost = tf.reduce_mean(tf.losses.sigmoid_cross_entropy(labels, logits))
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                ### END CODE HERE ###
                
                cost_per_iteration.append(minibatch_cost / minibatch_size)
                
                epoch_cost += minibatch_cost / minibatch_size

            # Print the cost every epoch
            if print_cost == True: #and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            #if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('epochs')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
        
        # plot the cost per iteration
        plt.plot(np.squeeze(cost_per_iteration)[:420])
        plt.ylabel('cost_per_iteration')
        plt.xlabel('iterations')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.math.round(Z3), Y)

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        # Попытка вычислить ROC-AUC
#         m = tf.keras.metrics.AUC()
#         m.update_state(Y_test, Z3)
#         print("ROC-AUC:", m.result())
        
        return parameters

In [ ]:
parameters = model(X_train, y_train, X_test, y_test)

# Построение модели в Keras

In [ ]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

In [ ]:
def model_keras(input_shape):
    """
    input_shape: The height, width and channels as a tuple.  
        Note that this does not include the 'batch' as a dimension.
        If you have a batch like 'X_train', 
        then you can provide the input_shape using
        X_train.shape[1:]
    """
 
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)
 
    # Hidden layer 1
    X = Dense(64, activation='relu', name='layer1')(X_input)
    
    # Hidden layer 2
    X = Dense(32, activation='relu', name='layer2')(X)
 
    # Output layer
    X = Dense(1, activation='sigmoid', name='output')(X)
 
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
 
    return model

In [ ]:
X_train.shape

In [ ]:
happyModel = model_keras(X_train.T.shape[1:])
happyModel.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
happyModel.fit(x = X_train.T, y = y_train.T, epochs = 15, batch_size = 1000)

In [ ]:
preds = happyModel.evaluate(x = X_test.T, y = y_test.T)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
happyModel.summary()

In [ ]:
plot_model(happyModel, to_file='HappyModel.png')
SVG(model_to_dot(happyModel).create(prog='dot', format='svg'))